In [1]:
!pip install efficientnet
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from efficientnet.keras import EfficientNetB5
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from sklearn.metrics import classification_report

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.1 MB/s eta 0:00:00


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
# Set the path to the dataset
data_path = '/kaggle/input/garbage-classification/garbage_classification'

def load_images(path, target_shape):
    X = []
    y = []
    for folder in os.listdir(path):
        if os.path.isdir(os.path.join(path, folder)):
            folder_path = os.path.join(path, folder)
            for image_file in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_file)
                image = cv2.imread(image_path)
                resized_image = cv2.resize(image, target_shape[:2])
                X.append(resized_image)
                y.append(folder)
    X = np.array(X)
    y = np.array(y)
    return X, y

# Set the target shape for input images
target_shape = (90, 90, 3)

# Load and preprocess the data
X, y = load_images(data_path, target_shape)
num_classes = len(np.unique(y))
y = to_categorical(LabelEncoder().fit_transform(y))

# Split the dataset into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
    

In [4]:
# Construct the EfficientNetB5 model
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=target_shape, pooling='max')

model = Sequential([
    base_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(256, kernel_regularizer=regularizers.l2(l=0.016), activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dropout(rate=0.45, seed=123),
    Dense(num_classes, activation='softmax')
])

model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

115515256/115515256 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-b5 (Functional  (None, 2048)             28513520  
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 12)                3084      
                                                       

In [5]:
import tensorflow as tf

class MyCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, patience, stop_patience, threshold, factor, batches, epochs, ask_epoch):
        super(MyCallback, self).__init__()
        self.model = model
        self.patience = patience
        self.stop_patience = stop_patience
        self.threshold = threshold
        self.factor = factor
        self.batches = batches
        self.epochs = epochs
        self.ask_epoch = ask_epoch
        self.ask_epoch_initial = ask_epoch

        self.count = 0
        self.stop_count = 0
        self.best_epoch = 1
        self.initial_lr = float(tf.keras.backend.get_value(model.optimizer.lr))
        self.highest_tracc = 0.0
        self.lowest_vloss = np.inf
        self.best_weights = self.model.get_weights()
        self.initial_weights = self.model.get_weights()

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.batches == 0 and epoch > 0:
            tr_acc = logs.get('accuracy')
            v_loss = logs.get('val_loss')

            if tr_acc > self.highest_tracc:
                self.highest_tracc = tr_acc

            if v_loss < self.lowest_vloss:
                self.lowest_vloss = v_loss
                self.best_epoch = epoch + 1
                self.best_weights = self.model.get_weights()
                self.count = 0
            else:
                self.count += 1

            if tr_acc >= self.threshold:
                self.ask_epoch -= 1

            if self.count == self.patience and self.ask_epoch > 0:
                print("\nEpoch %d: Accuracy threshold reached. Decreasing learning rate." % (epoch + 1))
                old_lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
                new_lr = old_lr * self.factor
                tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)
                print("Learning rate decreased from %f to %f" % (old_lr, new_lr))
                self.count = 0

            if self.count == self.patience and self.ask_epoch == 0:
                self.stop_count += 1
                if self.stop_count == self.stop_patience:
                    print("\nTraining stopped at epoch %d" % (epoch + 1))
                    self.model.stop_training = True
                else:
                    print("\nEpoch %d: Learning rate adjustment limit reached. Restoring best weights." % (epoch + 1))
                    self.model.set_weights(self.best_weights)
                    self.count = 0

In [15]:
# Define the callbacks
checkpoint = ModelCheckpoint('/kaggle/working/best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
my_callback = MyCallback(model=model, patience=10, stop_patience=5, threshold=0.9, factor=0.1, batches=10, epochs=100, ask_epoch=10)

# Train the model
history = model.fit(X_train, y_train, batch_size=8, epochs=20, validation_data=(X_val, y_val), callbacks=[checkpoint, my_callback])

# Load the best weights
model.load_weights('/kaggle/working/best_model.h5')

Epoch 1/20
1397/1397 [==============================] - ETA: 0s - loss: 0.2296 - accuracy: 0.9870
Epoch 1: val_accuracy improved from -inf to 0.93398, saving model to /kaggle/working/best_model.h5
1397/1397 [==============================] - 144s 103ms/step - loss: 0.2296 - accuracy: 0.9870 - val_loss: 0.4337 - val_accuracy: 0.9340
Epoch 2/20
1397/1397 [==============================] - ETA: 0s - loss: 0.2284 - accuracy: 0.9855
Epoch 2: val_accuracy improved from 0.93398 to 0.93639, saving model to /kaggle/working/best_model.h5
1397/1397 [==============================] - 145s 104ms/step - loss: 0.2284 - accuracy: 0.9855 - val_loss: 0.4037 - val_accuracy: 0.9364
Epoch 3/20
1397/1397 [==============================] - ETA: 0s - loss: 0.2135 - accuracy: 0.9876
Epoch 3: val_accuracy did not improve from 0.93639
1397/1397 [==============================] - 140s 100ms/step - loss: 0.2135 - accuracy: 0.9876 - val_loss: 0.4118 - val_accuracy: 0.9332
Epoch 4/20
1397/1397 [=====================

In [14]:
# Generate predictions for the test set
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

# Convert integer labels to class names
class_names = sorted(os.listdir(data_path))
y_true_names = np.array(class_names)[y_true_labels]
y_pred_names = np.array(class_names)[y_pred_labels]

# Generate and print the classification report
report = classification_report(y_true_names, y_pred_names)
print("Classification Report:")
print(report)


97/97 [==============================] - 4s 37ms/step
Classification Report:
              precision    recall  f1-score   support

     battery       0.90      0.96      0.93       188
  biological       0.93      0.96      0.94       196
 brown-glass       0.95      0.90      0.92       135
   cardboard       0.96      0.90      0.93       186
     clothes       0.99      0.98      0.99      1076
 green-glass       0.97      0.93      0.95       119
       metal       0.89      0.84      0.87       159
       paper       0.92      0.96      0.94       204
     plastic       0.86      0.80      0.83       157
       shoes       0.93      0.96      0.94       394
       trash       0.94      0.94      0.94       142
 white-glass       0.86      0.92      0.89       147

    accuracy                           0.94      3103
   macro avg       0.92      0.92      0.92      3103
weighted avg       0.94      0.94      0.94      3103



In [17]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)






97/97 [==============================] - 4s 42ms/step - loss: 0.3691 - accuracy: 0.9368
Test loss: 0.3690968155860901
Test accuracy: 0.9368353486061096
